In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


## Reading the File

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding = 'utf-8')

In [4]:
# length of the text is the number of characters in it
print(f"Length of text: {len(text)} characters")

Length of text: 1115394 characters


In [5]:
# Peeking at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
# Unique characters in the file
vocab = sorted(set(text))
print(f"{len(vocab)} unique characters")

65 unique characters


## Processing the Text

Before training, you need to convert the strings to a numerical representation.

### The preprocessing.
`StringLookup` layer can convert each character into a numeric ID.   
It just needs the text to be split into tokens first.

In [7]:
example_texts = ['example', 'ghi']
chars = tf.strings.unicode_split(example_texts, input_encoding = 'UTF-8')
chars

<tf.RaggedTensor [[b'e', b'x', b'a', b'm', b'p', b'l', b'e'], [b'g', b'h', b'i']]>

Creating a preprocessing lookup layer

In [8]:
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [9]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[44, 63, 40, 52, 55, 51, 44], [46, 47, 48]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it.  
For this you can use `preprocessing.StringLookup(..., invert=True)`.

In [10]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary = ids_from_chars.get_vocabulary(), invert=True, mask_token=None
)

In [11]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'e', b'x', b'a', b'm', b'p', b'l', b'e'], [b'g', b'h', b'i']]>

You can `tf.strings.reduce_join` to join the characters back into strings.

In [12]:
tf.strings.reduce_join(chars, axis = -1).numpy()

array([b'example', b'ghi'], dtype=object)

In [13]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis = -1)

### The prediction task
Given a character, or a sequence of characters, what is the most probable next character?  
This is the task you're training the model to perform.  
The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?

### Create training examples and targets
Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.  

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.  

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".  

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.  




In [14]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [15]:
ids_dataset=  tf.data.Dataset.from_tensor_slices(all_ids)

In [16]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [17]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length + 1)

The batch method lets you easily convert these individual characters to sequences of the desired size.

In [18]:
sequences = ids_dataset.batch(seq_length + 1, drop_remainder = True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [19]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


For training you'll need a dataset of (input, label) pairs. Where input and label are sequences. At each time step the input is the current character and the label is the next character.  

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:




In [20]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [21]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [22]:
dataset = sequences.map(split_input_target)

In [23]:
for input_example, target_example in dataset.take(1):
    print("Input: ", text_from_ids(input_example).numpy())
    print("Target: ", text_from_ids(target_example).numpy())

Input:  b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target:  b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


### Create training batches
You used `tf.data` to split the text into manageable sequences.  
But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [24]:
# batch size 
BATCH_SIZE = 64

# buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer  in which it shuffles elements.)
BUFFER_SIZE = 10000

dataset = (dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True).prefetch(tf.data.experimental.AUTOTUNE))
dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

### Build The Model
This section defines the model as a keras.Model subclass (For details see Making new Layers and Models via subclassing).  

This model has three layers:  

`tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with embedding_dim dimensions;   
`tf.keras.layers.GRU`: A type of RNN with size units=rnn_units (You can also use an LSTM layer here.)  
`tf.keras.layers.Dense`: The output layer, with vocab_size outputs. It outputs one logit for each character in the vocabulary.   
These are the log-likelihood of each character according to the model.  

In [25]:
# length of the vocabulary in characters
vocab_size = len(vocab)

# the embedding dimension
embedding_dim = 256

# number of RNN units
rnn_units = 1024

In [26]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                        return_sequences = True,
                                        return_state = True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states = None, return_state = False, training = False):
        x = inputs
        x = self.embedding(x, training = training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training = training)
        x = self.dense(x, training = training)

        if return_state:
            return x, states
        else:
            return x

In [27]:
model = MyModel(
    # the vocabulary size has to match the 'StringLookup' layers.
    vocab_size = len(ids_from_chars.get_vocabulary()),
    embedding_dim = embedding_dim, 
    rnn_units = rnn_units
)

## #Try the model
Now run the model to see that it behaves as expected.

First check the shape of the output:

In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [29]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  16896     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  67650     
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [30]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples = 1)
sampled_indices = tf.squeeze(sampled_indices, axis = -1).numpy()

In [31]:
sampled_indices

array([31, 49, 18, 47,  2, 60, 15, 42,  9, 61, 25, 54, 31, 29,  6, 41, 17,
       55, 15, 46, 44, 59, 24, 40, 48, 10, 51, 37, 18, 48, 21, 49, 48, 26,
        6, 30, 40, 39,  7, 53, 62,  3, 35, 27, 63, 24, 59, 35, 12, 36,  6,
       62, 10,  9, 32, 17, 23, 57,  4, 28, 22, 13, 33, 54,  6,  2, 30, 43,
       19, 27, 64, 21, 56, 60, 21, 11, 38, 55, 42, 56, 19,  4,  6, 52, 27,
       26,  8, 59,  3, 47,  8, 33, 50, 25, 34, 55, 58, 37,  8, 56])

Decode these to see the text predicted by this untrained model:

In [32]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'thy tongue detect thy base-born heart?\n\nEDWARD:\nA wisp of straw were worth a thousand crowns,\nTo mak'

Next Char Predictions:
 b"RjEh uBc.vLoRP'bDpBgetKai3lXEiHjiM'QaZ,nw!VNxKtV;W'w3.SDJr$OI?To' QdFNyHquH:YpcqF$'mNM-t!h-TkLUpsX-q"


### Train the model
At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.  

Attach an optimizer, and a loss function  
The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.   

Because your model returns logits, you need to set the from_logits flag.

In [33]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [34]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.1907787


A newly initialized model shouldn't be too sure of itself,   
the output logits should all have similar magnitudes.   
To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size.  
A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [35]:
tf.exp(mean_loss).numpy()

66.07423

Configure the training procedure using the `tf.keras.Model.compile` method. Use tf.keras.optimizers.Adam with default arguments and the loss function.

In [36]:
model.compile(optimizer = 'adam', loss = loss)

### Configure checkpoints
Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:

In [37]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [38]:
EPOCHS = 20

In [39]:
history = model.fit(dataset, epochs = EPOCHS, callbacks = [checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 26s 131ms/step - loss: 2.7006
Epoch 2/20
172/172 [==============================] - 24s 133ms/step - loss: 1.9760
Epoch 3/20
172/172 [==============================] - 24s 133ms/step - loss: 1.6998
Epoch 4/20
172/172 [==============================] - 24s 133ms/step - loss: 1.5411
Epoch 5/20
172/172 [==============================] - 24s 132ms/step - loss: 1.4434
Epoch 6/20
172/172 [==============================] - 24s 132ms/step - loss: 1.3768
Epoch 7/20
172/172 [==============================] - 24s 132ms/step - loss: 1.3249
Epoch 8/20
172/172 [==============================] - 24s 132ms/step - loss: 1.2801
Epoch 9/20
172/172 [==============================] - 24s 132ms/step - loss: 1.2395
Epoch 10/20
172/172 [==============================] - 24s 132ms/step - loss: 1.2002
Epoch 11/20
172/172 [==============================] - 24s 133ms/step - loss: 1.1602
Epoch 12/20
172/172 [==============================] - 24s 132ms/step - lo

### Generate text
The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.  

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.

In [40]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # creating a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values = [-float('inf')] * len(skip_ids),
            indices = skip_ids,
            # Match the shape to the vocabulary
            dense_shape = [len(ids_from_chars.get_vocabulary())]
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states = None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(inputs=input_ids, states=states,
                                            return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [41]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary.   
With the small number of training epochs, it has not yet learned to form coherent sentences.

In [42]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Go to, in heavens! when I commends this,
That fresh unwillingned, penitent he speak
Main their scars, did ever be found?
It shallent do me gracious instal;
There shall you hence before he of their deaths.
I'll gevorted me by your sides legs desparate;
And, if you wish me here behind your brother;
And there, thou draw'st, I apply so well:
He is nothing to bed what is the rest,
Enforce thou use hugh spoke, make her spoke of.

First Citizen:
Where is thy passago! the name of it!

ARIEL:
No, as I am, you;--perhaps,
Aleay'st Prince Edward, that she speaks:
Whither wilt not prove a waked by the day!

BALTHASAR:
How far off, lords, down forth from me abood
That will no more dishinging.

AUTOLYCUS:
Come, quick from my goods.

First Watchman:
We are all undonery? O, they had much:
And weep shall be dead.

SIRININ ELIZABETH:
Partly, for he hath warder than the suburbs,
Volscians actions patience to-day:
That libents that I have to no suborn,
I was born to privilege and free
And from my he

The easiest thing you can do to improve the results is to train it for longer (try EPOCHS = 30).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text faster the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above.

In [44]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nYour mis-shanks you go?\n\nMENENIUS:\nThere was some signeth doth be satisfied.\n\nGEORGE:\nOur general ill!\nIntended youth-fasliar was?\n\nLUCENTIO:\nI fear, I pray, he ne'er been said,\nAnd perisheet so much in your cold extreme.\n\nQUEEN ELIZABETH:\nThe sad, he prizes night by the other.\nO, never fault, and better what this meening?\nI long to hear them for the place, or happy state,\nEre he nothing like to his news.\n\nLEONTES:\nNorning, be content.\n\nHENRY BOLINGBROKE:\nEven we dread upon this price, at less banished age\ntill he indeed,\nAnd thereto kingly said we parted up, at move\nMy father gracious in the hand of heaven.\n\nProvost:\nThis is an ass' humbly shall absolute gentlemen:\nThis did have made thy father's bosom there,\nThat want at two more sons, which in these unproshes\nWhat's angry that we are then; but he's more, his ungent.\n\nKING RICHARD III:\nBid my hand I ne'er was married to Mercury?\nThere is audoriced corn! would buy the\nbattle; 

The full tutorial https://www.tensorflow.org/text/tutorials/text_generation